In [938]:
import pandas as pd
import numpy as np

In [1017]:
states = np.array([0, 1])
observations= np.array([1,1,2,1,0,1,2,1,0,2,2,0,1,0,1])
start_probabilities = np.array([0.5, 0.5])
transition_probabilities = np.array([[0.7, 0.3],
                                     [0.4, 0.6]])
emission_probabilities = np.array([[0.2, 0.6, 0.2],
                                   [0.4, 0.1, 0.5]])

In [1019]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    n_obs = len(obs)
    n_states = len(states)
    
    out = np.zeros((n_states, n_obs))
    paths = np.zeros(n_obs)
    phi = np.zeros((n_states, n_obs))
    
    for t in range(n_obs):
        for s in states:
            if t == 0:
                out[s, 0] = start_p[s] * emit_p[s, obs[0]]
            else:
                out[s, t] = np.max(out[:, t - 1] * emit_p[s, obs[t]] * trans_p[:, s])
                phi[s, t] = np.argmax(out[:, t - 1] * trans_p[:, s])

    paths[n_obs - 1] = np.argmax(out[:, n_obs - 1])
    for o in range(n_obs - 2, -1, -1):
        paths[o] = phi[int(paths[o + 1]), [o + 1]]

    return out, paths

out, paths = viterbi(observations, states, start_probabilities, transition_probabilities, emission_probabilities)

print(out)

obs = ['eating', 'eating', 'pooping', 'eating', 'sleeping', 'eating', 'pooping', 'eating', 'sleeping', 'pooping', 'pooping', 'sleeping', 'eating', 'sleeping', 'eating']
pd.DataFrame([(obs[idx], ['healthy', 'sick'][int(p)]) for idx, p in enumerate(paths)])

[[  3.00000000e-01   1.26000000e-01   1.76400000e-02   7.40880000e-03
    1.03723200e-03   4.35637440e-04   6.09892416e-05   2.56154815e-05
    3.58616741e-06   5.02063437e-07   7.37725866e-08   2.21317760e-08
    1.59348787e-08   2.23088302e-09   9.36970868e-10]
 [  5.00000000e-02   9.00000000e-03   1.89000000e-02   1.13400000e-03
    8.89056000e-04   5.33433600e-05   6.53456160e-05   3.92073696e-06
    3.07385778e-06   9.22157333e-07   2.76647200e-07   6.63953280e-08
    3.98371968e-09   1.91218545e-09   1.14731127e-10]]


,0,1
0,eating,healthy
1,eating,healthy
2,pooping,healthy
3,eating,healthy
4,sleeping,healthy
5,eating,healthy
6,pooping,healthy
7,eating,healthy
8,sleeping,sick
9,pooping,sick


In [1021]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0][st] = {'prob': start_p[st] * emit_p[st][obs[0]], 'prev': None}
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = max(V[t-1][prev_st]['prob']*trans_p[prev_st][st] for prev_st in states)
            for prev_st in states:
                if V[t-1][prev_st]['prob'] * trans_p[prev_st][st] == max_tr_prob:
                    max_prob = max_tr_prob * emit_p[st][obs[t]]
                    V[t][st] = {'prob': max_prob, 'prev': prev_st}
                    break
    for line in dptable(V):
        print(line)
    opt = []
    # The highest probability
    max_prob = max(value['prob'] for value in V[-1].values())
    previous = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data['prob'] == max_prob:
            opt.append(st)
            previous = st
            break
    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]['prev'])
        previous = V[t + 1][previous]['prev']

    print('The steps of states are ' + ' '.join(opt) + ' with highest probability of %s' % max_prob)

def dptable(V):
#     Print a table of steps from dictionary
    yield " ".join(("%12d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)

In [1022]:
obs = ('eating', 'eating', 'pooping', 'eating', 'sleeping', 'eating', 'pooping', 'eating', 'sleeping', 'pooping', 'pooping', 'sleeping', 'eating', 'sleeping', 'eating')
states = ('healthy', 'sick')

start_p = {'healthy': 0.5, 'sick': 0.5}
trans_p = {'healthy': {'healthy': 0.7, 
                                        'sick': 0.3},
                            'sick': {'healthy': 0.4, 
                                      'sick': 0.6}}
emit_p = {'healthy': {'sleeping': 0.2, 
                                      'eating': 0.6, 
                                      'pooping': 0.2},
                          'sick': {'sleeping': 0.4, 
                                    'eating': 0.1, 
                                 'pooping': 0.5}}
viterbi(obs, states, start_p, trans_p, emit_p)

           0            1            2            3            4            5            6            7            8            9           10           11           12           13           14
healthy: 0.30000 0.12600 0.01764 0.00740 0.00103 0.00043 0.00006 0.00002 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000
sick: 0.05000 0.00900 0.01890 0.00113 0.00088 0.00005 0.00006 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000
The steps of states are healthy healthy healthy healthy healthy healthy healthy healthy sick sick sick sick healthy healthy healthy with highest probability of 9.369708683891833e-10


In [1025]:
obs = ('normal', 'cold', 'dizzy')
states = ('Healthy', 'Fever')
start_p = {'Healthy': 0.6, 'Fever': 0.4}
trans_p = {
   'Healthy' : {'Healthy': 0.7, 'Fever': 0.3},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.6}
   }
emit_p = {
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6}
   }
viterbi(obs, states, start_p, trans_p, emit_p)

           0            1            2
Healthy: 0.30000 0.08400 0.00588
Fever: 0.04000 0.02700 0.01512
The steps of states are Healthy Healthy Fever with highest probability of 0.01512
